In [3]:
import warnings
warnings.filterwarnings('ignore')

import sys
import random
import numpy as np
import pyodbc 
from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

import pandas as pd
import matplotlib.pyplot as plt


In [4]:
#Conectando com o Impala
pyodbc.autocommit = True
conn_impala = pyodbc.connect('DSN=Sample Cloudera Impala DSN 64', autocommit=True)
cursor_impala = conn_impala.cursor()

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Nome da fonte de dados não encontrado e nenhum driver padrão especificado (0) (SQLDriverConnect)')

In [ ]:
cursor_impala.execute('show databases; ')

In [ ]:
def as_pandas(cursor_impala):
    names = [metadata[0] for metadata in cursor_impala.description]
    return pd.DataFrame([dict(zip(names, row)) for row in cursor_impala], columns=names)

In [ ]:
as_pandas(cursor_impala)

In [ ]:
cursor_impala.execute('use voos_brasil_labdata; ')

In [ ]:
cursor_impala.execute('show tables; ')
as_pandas(cursor_impala)

In [ ]:
cursor_impala.execute('select * from voos_historico_tratada_5 where situacao_voo = "REALIZADO"')# where situacao_voo in (%s)', (['REALIZADO'],))

In [5]:
dataset = pd.read_csv('voos.csv', encoding='utf-8', sep=',')#as_pandas(cursor_impala)

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
dataset

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero
0,GLO,1147,0,N,SBCT,AFONSO PENA,S�O JOS� DOS PINHAIS,PR,BRASIL,AMERICA DO SUL,SBCT,908.0,32.0,909.1,19.0,18.2,92.0,3.0,NW,SBSP,CONGONHAS,S�O PAULO,SP,BRASIL,AMERICA DO SUL,2017-04-21 08:40:00,2017,4,21,2017-04-21,840,2017-04-21 08:40:00,2017.0,4.0,21.0,SBSP,803.0,14.0,923.9,20.6,18.5,82.0,3.0,SE,2017-04-21 07:45:00,2017.0,4.0,21.0,2017-04-21,840,2017-04-21 07:45:00,REALIZADO,NaN,0,0.0,0,0.0,1,1,0,0,0,0,0,0,0,0,4,4,1,1,908,32,909.1,19.0,18.2,92,3,803,14,923.9,20.6,18.5,82,3,908,32,909.1,19.0,18.2,92.0,3.0,803.0,14.0,923.9,20.6,18.5,82.0,3.0,908,32,909.1,19.0,18.2,92,3,803,14,923.9,20.6,18.5,82,3
1,GLO,1141,0,R,SBCT,AFONSO PENA,S�O JOS� DOS PINHAIS,PR,BRASIL,AMERICA DO SUL,SBCT,908.0,32.0,909.1,19.0,18.2,92.0,3.0,NW,SBSP,CONGONHAS,S�O PAULO,SP,BRASIL,AMERICA DO SUL,2017-04-21 14:40:00,2017,4,21,2017-04-21,1440,2017-04-21 14:40:00,2017.0,4.0,21.0,SBSP,803.0,14.0,923.9,20.6,18.5,82.0,3.0,SE,2017-04-21 13:40:00,2017.0,4.0,21.0,2017-04-21,1440,2017-04-21 13:40:00,REALIZADO,NaN,0,0.0,0,0.0,1,1,0,0,0,0,0,0,0,0,4,4,1,1,908,32,909.1,19.0,18.2,92,3,803,14,923.9,20.6,18.5,82,3,908,32,909.1,19.0,18.2,92.0,3.0,803.0,14.0,923.9,20.6,18.5,82.0,3.0,908,32,909.1,19.0,18.2,92,3,803,14,923.9,20.6,18.5,82,3
2,GLO,1139,0,N,SBCT,AFONSO PENA,S�O JOS� DOS PINHAIS,PR,BRASIL,AMERICA DO SUL,SBCT,908.0,32.0,909.1,19.0,18.2,92.0,3.0,NW,SBSP,CONGONHAS,S�O PAULO,SP,BRASIL,AMERICA DO SUL,2017-04-21 12:45:00,2017,4,21,2017-04-21,1245,2017-04-21 12:45:00,2017.0,4.0,21

In [8]:
# PANDAS: Verifica a dimensão dos dados (linhas, colunas)
dim = dataset.shape


In [9]:
dim

(99999, 113)

In [10]:
# PANDAS: Verifica os tipos de dados de cada atributo
tipos = dataset.dtypes
print(tipos)

icao                                         object
num_voo                                       int64
cod_aut_di                                   object
cod_tipo_linha                               object
cod_aero_org                                 object
aeroporto_origem                             object
cidade_origem                                object
uf_origem                                    object
pais_origem                                  object
continente_origem                            object
icao_origem                                  object
altitudeicao_origem                         float64
direcaovento_origem                         float64
pressaoatmestacao_origem                    float64
tempbulboseco_origem                        float64
tempbulboumido_origem                       float64
umidaderelativa_origem                      float64
velocidadeventonebulosidade_origem          float64
descricao_cod_origem                         object
cod_aero_des

In [11]:
# PANDAS: Estatística descritiva
pd.set_option('display.width', 100)
pd.set_option('precision', 15)
resultado = dataset.describe()
print(resultado)

                     num_voo    altitudeicao_origem    direcaovento_origem  \
count  99999.000000000000000  94888.000000000000000  97868.000000000000000   
mean    3866.030440304402873    449.820693870668606     12.903676380430785   
std     2151.019296634482998    381.994768876795092     11.472085428708519   
min        1.000000000000000      1.000000000000000      0.000000000000000   
25%     2252.000000000000000     19.000000000000000      5.000000000000000   
50%     3362.000000000000000    612.000000000000000      9.000000000000000   
75%     5180.000000000000000    803.000000000000000     23.000000000000000   
max     9999.000000000000000    943.000000000000000     36.000000000000000   

       pressaoatmestacao_origem   tempbulboseco_origem  tempbulboumido_origem  \
count     95467.000000000000000  98882.000000000000000  85076.000000000000000   
mean        954.579013690714305     23.574379563520484     19.412686304012759   
std          44.014452680322812      5.275266566322420

In [12]:
dataset.head(10)

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero
0,GLO,1147,0,N,SBCT,AFONSO PENA,S�O JOS� DOS PINHAIS,PR,BRASIL,AMERICA DO SUL,SBCT,908.0,32.0,909.100000000000023,19.0,18.199999999999999,92.0,3.0,NW,SBSP,CONGONHAS,S�O PAULO,SP,BRASIL,AMERICA DO SUL,2017-04-21 08:40:00,2017,4,21,2017-04-21,840,2017-04-21 08:40:00,2017.0,4.0,21.0,SBSP,803.0,14.0,923.899999999999977,20.600000000000001,18.5,82.0,3.0,SE,2017-04-21 07:45:00,2017.0,4.0,21.0,2017-04-21,840,2017-04-21 07:45:00,REALIZADO,NaN,0,0.0,0,0.0,1,1,0,0,0,0,0,0,0,0,4,4,1,1,908,32,909.100000000000023,19.0,18.199999999999999,92,3,803,14,923.899999999999977,20.600000000000001,18.5,82,3,908,32,909.100000000000023,19.0,18.199999999999999,92.0,3.0,803.0,14.0,923.899999999999977,20.600000000000001,18.5,82.0,3.0,908,32,909.100000000000023,19.0,18.199999999999999,92,3,803,14,923.899999999999977,20.600000000000001,18.5,82,3
1,GLO,1141,0,R,SBCT,AFONSO PENA,S�O JOS� DOS PINHAIS,PR,BRASIL,AMERICA DO SUL,SBCT,908.0,32.0,909.100000000000023,19.0,18.199999999999999,92.0,3.0,NW,SBSP,CONGONHAS,S�O PAULO,SP,BRASIL,AMERICA DO SUL,2017-04-21 14:40:00,2017,4,21,2017-04-21,1440,2017-04-21 14:40:00,2017.0,4.0,21.0,SBSP,803.0,14.0,923.899999999999977,20.600000000000001,18.5,82.0,3.0,SE,2017-04-21 13:40:00,2017.0,4.0,21.0,2017-04-21,1440,2017-04-21 13:40:00,REALIZADO,NaN,0,0.0,0,0.0,1,1,0,0,0,0,0,0,0,0,4,4,1,1,908,32,909.100000000000023,19.0,18.199999999999999,92,3,803,14,923.899999999999977,20.600000000000001,18.5,82,3,908,32,909.100000000000023,19.0,18.1999

In [13]:
#TRATAMENTOS DE DADO
dataset_filtrado = dataset.loc[dataset['situacao_voo'] == 'REALIZADO']

In [14]:
#DUMMIES
dummies_aero_org = pd.get_dummies(dataset_filtrado['cod_aero_org'], prefix='dummy_origem')
dummies_aero_dest = pd.get_dummies(dataset_filtrado['cod_aero_dest'], prefix='dummy_destino')

colunas_dummie_aero_org = list(dummies_aero_org)
colunas_dummie_aero_dest = list(dummies_aero_dest)

dataset_filtrado = pd.concat([dataset_filtrado, dummies_aero_org], axis=1)
dataset_filtrado = pd.concat([dataset_filtrado, dummies_aero_dest], axis=1)

In [15]:
dataset_filtrado

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero,dummy_origem_SBAR,dummy_origem_SBBR,dummy_origem_SBBV,dummy_origem_SBCF,dummy_origem_SBCT,dummy_origem_SBDN,dummy_origem_SBFL,dummy_origem_SBFN,dummy_origem_SBGL,dummy_origem_SBGR,dummy_origem_SBIH,dummy_origem_SBKP,dummy_origem_SBLE,dummy_origem_SBMA,dummy_origem_SBMD,dummy_origem_SBMG,dummy_origem_SBMQ,dummy_origem_SBMY,dummy_origem_SBOI,dummy_origem_SBPA,dummy_origem_SBPK,dummy_origem_SBPS,dummy_origem_SBRB,dummy_origem_SBRF,dummy_origem_SBRJ,dummy_origem_SBRP,dummy_origem_SBSL,dummy_origem_SBSP,dummy_origem_SBSV,dummy_origem_SBTC,dummy_origem_SBTT,dummy_origem_SBUL,dummy_origem_SBUY,dummy_origem_SBVH,dummy_origem_SWPI,dummy_destino_SBAR,dummy_destino_SBAT,dummy_destino_SBAU,dummy_destino_SBBE,dummy_destino_SBBH,dummy_destino_SBBR,dummy_destino_SBBV,dummy_destino_SBCA,dummy_destino_SBCB,dummy_destino_SBCF,dummy_destino_SBCG,dummy_destino_SBCH,dummy_destino_SBCJ,dummy_destino_SBCM,dummy_destino_SBCN,dummy_destino_SBCP,dummy_destino_SBCR,dummy_destino_SBCT,dummy_destino_SBCX,dummy_destino_SBCY,dummy_destino_SBCZ,dummy_destino_SBDN,dummy_destino_SBEG,dummy_destino_SBFI,dummy_destino_SBFL,dummy_destino_SBFN,dummy_destino_SBFZ,dummy_destino_SBGL,dummy_destino_SBGO,dummy_destino_SBGR,dummy_destino_SBGV,dummy_destino_SBHT,dummy_destino_SBIH,dummy_destino_SBIL,dummy_destino_SBIP,dummy_destino_SBIZ,dummy_destino_SBJP,dummy_destino_SBJU,dummy_destino_SBJV,dummy_destino_SBKG,dummy_destino_SBKP,dummy_destino_SBLE,dummy_destino_SBLJ,dummy_dest

In [16]:
# PANDAS: Verifica a dimensão dos dados (linhas, colunas)
dim = dataset_filtrado.shape
print(dim)

(88598, 235)


In [17]:
# PANDAS: Estatística descritiva
pd.set_option('display.width', 100)
pd.set_option('precision', 15)
resultado = dataset_filtrado.describe()
print(resultado)

                     num_voo    altitudeicao_origem    direcaovento_origem  \
count  88598.000000000000000  83965.000000000000000  86782.000000000000000   
mean    3934.574008442628383    455.226737331030790     12.982865110276325   
std     2193.157717884067551    380.937587992577903     11.451482060927983   
min        1.000000000000000      1.000000000000000      0.000000000000000   
25%     2271.000000000000000     19.000000000000000      5.000000000000000   
50%     3445.000000000000000    621.000000000000000      9.000000000000000   
75%     5321.000000000000000    803.000000000000000     23.000000000000000   
max     9999.000000000000000    943.000000000000000     36.000000000000000   

       pressaoatmestacao_origem   tempbulboseco_origem  tempbulboumido_origem  \
count     84258.000000000000000  87584.000000000000000  75520.000000000000000   
mean        953.962394075420775     23.557105179028632     19.375417108052250   
std          43.811917603217758      5.243563626616845

In [18]:
# PANDAS: Distribuição das classses
contagem = dataset_filtrado.groupby('flg_atraso_partida').size()
print(contagem)

flg_atraso_partida
0    79475
1     9123
dtype: int64


In [19]:
# PANDAS: Correlação
cor = dataset_filtrado.corr(method='pearson')
print(cor)

                                              num_voo  altitudeicao_origem  direcaovento_origem  \
num_voo                             1.000000000000000   -0.126073313908716   -0.022263737668531   
altitudeicao_origem                -0.126073313908716    1.000000000000000    0.272310058953016   
direcaovento_origem                -0.022263737668531    0.272310058953016    1.000000000000000   
pressaoatmestacao_origem            0.139525572509500   -0.649515202995567   -0.130675357875862   
tempbulboseco_origem                0.063074421379856   -0.366917837656499   -0.016576813246323   
tempbulboumido_origem               0.007528155104743   -0.465622231929778   -0.106261314451271   
umidaderelativa_origem              0.046611018355228   -0.145633759381822   -0.227371938089171   
velocidadeventonebulosidade_origem -0.055121087061700    0.205919758252740    0.363041644757472   
chegada_prevista_ano                0.077417097898236   -0.025596274748670    0.062677775403089   
chegada_pr

In [20]:
dataset_filtrado.shape

(88598, 235)

In [21]:
dataset_2017 = dataset_filtrado.loc[dataset_filtrado['partida_prevista_ano'] == 2017]
dataset_2018 = dataset_filtrado.loc[dataset_filtrado['partida_prevista_ano'] == 2018]


In [22]:
dataset_2017.shape

(22258, 235)

In [23]:
dataset_2018.shape

(6631, 235)

In [24]:
dataset_2017

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero,dummy_origem_SBAR,dummy_origem_SBBR,dummy_origem_SBBV,dummy_origem_SBCF,dummy_origem_SBCT,dummy_origem_SBDN,dummy_origem_SBFL,dummy_origem_SBFN,dummy_origem_SBGL,dummy_origem_SBGR,dummy_origem_SBIH,dummy_origem_SBKP,dummy_origem_SBLE,dummy_origem_SBMA,dummy_origem_SBMD,dummy_origem_SBMG,dummy_origem_SBMQ,dummy_origem_SBMY,dummy_origem_SBOI,dummy_origem_SBPA,dummy_origem_SBPK,dummy_origem_SBPS,dummy_origem_SBRB,dummy_origem_SBRF,dummy_origem_SBRJ,dummy_origem_SBRP,dummy_origem_SBSL,dummy_origem_SBSP,dummy_origem_SBSV,dummy_origem_SBTC,dummy_origem_SBTT,dummy_origem_SBUL,dummy_origem_SBUY,dummy_origem_SBVH,dummy_origem_SWPI,dummy_destino_SBAR,dummy_destino_SBAT,dummy_destino_SBAU,dummy_destino_SBBE,dummy_destino_SBBH,dummy_destino_SBBR,dummy_destino_SBBV,dummy_destino_SBCA,dummy_destino_SBCB,dummy_destino_SBCF,dummy_destino_SBCG,dummy_destino_SBCH,dummy_destino_SBCJ,dummy_destino_SBCM,dummy_destino_SBCN,dummy_destino_SBCP,dummy_destino_SBCR,dummy_destino_SBCT,dummy_destino_SBCX,dummy_destino_SBCY,dummy_destino_SBCZ,dummy_destino_SBDN,dummy_destino_SBEG,dummy_destino_SBFI,dummy_destino_SBFL,dummy_destino_SBFN,dummy_destino_SBFZ,dummy_destino_SBGL,dummy_destino_SBGO,dummy_destino_SBGR,dummy_destino_SBGV,dummy_destino_SBHT,dummy_destino_SBIH,dummy_destino_SBIL,dummy_destino_SBIP,dummy_destino_SBIZ,dummy_destino_SBJP,dummy_destino_SBJU,dummy_destino_SBJV,dummy_destino_SBKG,dummy_destino_SBKP,dummy_destino_SBLE,dummy_destino_SBLJ,dummy_dest

In [72]:
#cols = ['direcaovento_origem','altitudeicao_origem','umidaderelativa_origem','velocidadeventonebulosidade_origem','chegada_prevista_ano','chegada_prevista_mes','chegada_prevista_dia','chegada_prevista_hora','chegada_real_ano','chegada_real_mes','chegada_real_dia','altitudeicao_dest','direcaovento_dest','umidaderelativa_dest','velocidadeventonebulosidade_dest','partida_prevista_ano','partida_prevista_mes','partida_prevista_dia','partida_prevista_hora']

#cols = ['feriado_federal_chegada', 'feriado_estadual_chegada', 'feriado_municipal_chegada','altitudeicao_origem_moda', 'direcaovento_origem_moda', 'pressaoatmestacao_origem_moda', 'tempbulboseco_origem_moda', 'tempbulboumido_origem_moda'] + colunas_dummie_aero_dest + colunas_dummie_aero_org

cols = ['feriado_federal_chegada', 'feriado_estadual_chegada', 'feriado_municipal_chegada', 'altitudeicao_origem_media', 'direcaovento_origem_media', 'pressaoatmestacao_origem_media', 'tempbulboseco_origem_media', 'tempbulboumido_origem_media'] + colunas_dummie_aero_dest + colunas_dummie_aero_org

y_total = dataset_filtrado['flg_atraso_partida']
x_total = dataset_filtrado[cols]

train_y =  dataset_2017['flg_atraso_partida']
train_x =  dataset_2017[cols]

test_y = dataset_2018['flg_atraso_partida']
test_x = dataset_2018[cols]

In [26]:
#REGRESSÃO LOGÍSTICA

clf_lr = linear_model.LogisticRegression(penalty='l2')
clf_lr.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
predictions = clf_lr.predict(test_x)

In [35]:
print(confusion_matrix(test_y,predictions))

[[5593    0]
 [1038    0]]


In [36]:
print(classification_report(test_y,predictions))

             precision    recall  f1-score   support

          0       0.84      1.00      0.92      5593
          1       0.00      0.00      0.00      1038

avg / total       0.71      0.84      0.77      6631



In [37]:
print(accuracy_score(test_y,predictions))

0.8434625245061077


In [73]:
n_estimators = [1, 5, 10, 15, 20, 25]

max_features = ['auto', 'sqrt']

max_depth = [10, 30, 50, 70, 100, 500, None]


min_samples_split = [2, 5, 10, 15, 20]

min_samples_leaf = [1, 2, 3, 5, 8, 13, 21, 34, 55]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [74]:
#RANDOMGRIDSEARCH RANDOM FOREST
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(x_total, y_total)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  9.6min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [1, 5, 10, 15, 20, 25], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 30, 50, 70, 100, 500, None], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 3, 5, 8, 13, 21, 34, 55], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [75]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 55,
 'min_samples_split': 15,
 'n_estimators': 25}

In [46]:
#RANDOM FOREST

rfc = RandomForestClassifier(max_depth=20, random_state=42)
rfc.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [47]:
predictions = rfc.predict(test_x)

In [48]:
print(confusion_matrix(test_y,predictions))

[[5590    3]
 [1037    1]]


In [49]:
print(classification_report(test_y,predictions))

             precision    recall  f1-score   support

          0       0.84      1.00      0.91      5593
          1       0.25      0.00      0.00      1038

avg / total       0.75      0.84      0.77      6631



In [50]:
print(accuracy_score(test_y,predictions))

0.8431609108731715


In [53]:
feature_importances = sorted(zip(map(lambda x: round(x, 4), rfc.feature_importances_), cols), reverse=True)
print (feature_importances)

[(0.2569, 'pressaoatmestacao_origem_media'), (0.2237, 'tempbulboseco_origem_media'), (0.176, 'tempbulboumido_origem_media'), (0.1012, 'direcaovento_origem_media'), (0.0148, 'altitudeicao_origem_media'), (0.0113, 'dummy_destino_SBBR'), (0.0109, 'dummy_destino_SBSP'), (0.0097, 'dummy_destino_SBGR'), (0.0088, 'feriado_federal_chegada'), (0.0087, 'dummy_destino_SBRF'), (0.0074, 'dummy_origem_SBKP'), (0.0072, 'dummy_destino_SBCF'), (0.0071, 'dummy_destino_SBKP'), (0.0071, 'dummy_destino_SBCT'), (0.0068, 'dummy_destino_SBFZ'), (0.0065, 'dummy_destino_SBQV'), (0.0063, 'dummy_origem_SBPS'), (0.0061, 'dummy_destino_SBPA'), (0.005, 'dummy_destino_SBPS'), (0.0044, 'dummy_destino_SBSG'), (0.0043, 'dummy_destino_SBFL'), (0.0039, 'dummy_destino_SBGO'), (0.0036, 'dummy_origem_SBSV'), (0.0034, 'dummy_destino_SBMG'), (0.0033, 'feriado_municipal_chegada'), (0.0033, 'dummy_origem_SBCT'), (0.0033, 'dummy_destino_SBJV'), (0.0032, 'dummy_destino_SBBE'), (0.0029, 'dummy_destino_SBMK'), (0.0029, 'dummy_destin